In [85]:
# Requirements

# From GeoNames (https://www.geonames.org/export/):
## cities15000.txt
## countryInfo.txt (https://download.geonames.org/export/dump/countryInfo.txt)

# %pip install pandas flashtext unidecode wordninja

In [86]:
# connect to mongoDB

from pymongo import MongoClient

MONGO_URI = "/" # connection URI here

client = MongoClient(MONGO_URI)

client.list_database_names()

['raw-scrape-data', 'admin', 'local']

In [87]:
# select database and collection

db = client["raw-scrape-data"]
collection = db["tiktok-user-data"]

collection.count_documents({})


7415

In [88]:
# load data into pandas (excluding the fields related to the scrape run)
# this is not limited - lading all records in

import pandas as pd

cursor = collection.find(
    {},
    {
        "_id": 0,
        "username": 1,
        "run_finished_at": 1,
        "video_id": 1,
        "caption": 1,
        "timestamp": 1,
        "duration_sec": 1,
        "view_count": 1,
        "like_count": 1,
        "comment_count": 1,
        "repost_count": 1,
        "hashtags": 1
    }
)

docs = list(cursor)
df = pd.DataFrame(docs)

df.head()

,run_finished_at,username,video_id,caption,timestamp,duration_sec,view_count,like_count,comment_count,repost_count,hashtags
0,2026-02-02 03:12:20.410,visitdubai,7601573860256894228,our beautiful beautiful city #Dubai #VisitDuba...,1769879340,21.0,1276,98,6,3,"dubai,visitdubai,fyp"
1,2026-02-02 03:12:20.410,visitdubai,7600024923301530900,200km/h through the sky. Ciel Dubai Marina jus...,1769518701,51.0,1175,48,2,2,"xdubai,visitdubai,dubai,thefirstgroup"
2,2026-02-02 03:12:20.410,visitdubai,7599717255353552148,Dubai but make it cinematic ❤️‍🔥 #VisitDubai #...,1769447068,15.0,1717,118,4,4,"visitdubai,dubai,fyp"
3,2026-02-02 03:12:20.410,visitdubai,7597750555636813076,2016 🤠 #VisitDubai #Dubai #fyp,1768989160,60.0,51200,5977,42,497,"visitdubai,dubai,fyp"
4,2026-02-02 03:12:20.410,visitdubai,7596781431444065556,lucky me ☺️ #VisitDubai #Dubai #fyp,1768763520,12.0,2215,109,5,5,"visitdubai,dubai,fyp"


In [89]:
# Convert timestamps to posted date/time

df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s", errors="coerce")
df.rename(columns={"timestamp": "posted_date_time"}, inplace=True)

df["run_finished_at"] = pd.to_datetime(df["run_finished_at"], unit="s", errors="coerce")
df.rename(columns={"run_finished_at": "scraped_at"}, inplace=True)

df.head()


,scraped_at,username,video_id,caption,posted_date_time,duration_sec,view_count,like_count,comment_count,repost_count,hashtags
0,2026-02-02 03:12:20.410,visitdubai,7601573860256894228,our beautiful beautiful city #Dubai #VisitDuba...,2026-01-31 17:09:00,21.0,1276,98,6,3,"dubai,visitdubai,fyp"
1,2026-02-02 03:12:20.410,visitdubai,7600024923301530900,200km/h through the sky. Ciel Dubai Marina jus...,2026-01-27 12:58:21,51.0,1175,48,2,2,"xdubai,visitdubai,dubai,thefirstgroup"
2,2026-02-02 03:12:20.410,visitdubai,7599717255353552148,Dubai but make it cinematic ❤️‍🔥 #VisitDubai #...,2026-01-26 17:04:28,15.0,1717,118,4,4,"visitdubai,dubai,fyp"
3,2026-02-02 03:12:20.410,visitdubai,7597750555636813076,2016 🤠 #VisitDubai #Dubai #fyp,2026-01-21 09:52:40,60.0,51200,5977,42,497,"visitdubai,dubai,fyp"
4,2026-02-02 03:12:20.410,visitdubai,7596781431444065556,lucky me ☺️ #VisitDubai #Dubai #fyp,2026-01-18 19:12:00,12.0,2215,109,5,5,"visitdubai,dubai,fyp"


In [90]:
# create a copy to preserve the raw data frame

df_raw = df.copy()
df = df_raw.copy()


In [91]:
# hashtags into a list for analysis later

df["hashtags_list"] = (
    df["hashtags"]
    .fillna("")
    .astype(str)
    .str.lower()
    .str.replace(r"\s+", "", regex=True)
    .str.split(",")
    .apply(lambda lst: [t for t in lst if t])
)

df.head()


,scraped_at,username,video_id,caption,posted_date_time,duration_sec,view_count,like_count,comment_count,repost_count,hashtags,hashtags_list
0,2026-02-02 03:12:20.410,visitdubai,7601573860256894228,our beautiful beautiful city #Dubai #VisitDuba...,2026-01-31 17:09:00,21.0,1276,98,6,3,"dubai,visitdubai,fyp","[dubai, visitdubai, fyp]"
1,2026-02-02 03:12:20.410,visitdubai,7600024923301530900,200km/h through the sky. Ciel Dubai Marina jus...,2026-01-27 12:58:21,51.0,1175,48,2,2,"xdubai,visitdubai,dubai,thefirstgroup","[xdubai, visitdubai, dubai, thefirstgroup]"
2,2026-02-02 03:12:20.410,visitdubai,7599717255353552148,Dubai but make it cinematic ❤️‍🔥 #VisitDubai #...,2026-01-26 17:04:28,15.0,1717,118,4,4,"visitdubai,dubai,fyp","[visitdubai, dubai, fyp]"
3,2026-02-02 03:12:20.410,visitdubai,7597750555636813076,2016 🤠 #VisitDubai #Dubai #fyp,2026-01-21 09:52:40,60.0,51200,5977,42,497,"visitdubai,dubai,fyp","[visitdubai, dubai, fyp]"
4,2026-02-02 03:12:20.410,visitdubai,7596781431444065556,lucky me ☺️ #VisitDubai #Dubai #fyp,2026-01-18 19:12:00,12.0,2215,109,5,5,"visitdubai,dubai,fyp","[visitdubai, dubai, fyp]"


In [92]:
# Remove duplicates

before = len(df) #curious how many are dropped

subset_cols = [c for c in ["video_id", "scraped_at", "url", "username"] if c in df.columns]
df = df.drop_duplicates(subset=subset_cols)

after = len(df)
print(f"Exact duplicates dropped (subset={subset_cols}): {before - after}") # exact dropped

before = len(df)

df = df.sort_values("scraped_at").drop_duplicates(subset=["video_id"], keep="last")

after = len(df)
print(f"Duplicate video_id rows dropped (kept latest scraped_at): {before - after}") # duplicate videos

Exact duplicates dropped (subset=['video_id', 'scraped_at', 'username']): 0
Duplicate video_id rows dropped (kept latest scraped_at): 90


In [93]:
# standardize numeric datatypes

metric_cols = ["view_count","like_count","comment_count","repost_count","save_count","duration","duration_sec"]
for c in metric_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# unify duration naming
if "duration_sec" in df.columns and "duration" not in df.columns:
    df.rename(columns={"duration_sec":"duration_s"}, inplace=True)
elif "duration" in df.columns:
    df.rename(columns={"duration":"duration_s"}, inplace=True)

# update null counts to 0
for c in ["view_count","like_count","comment_count","repost_count","save_count"]:
    if c in df.columns:
        df[c] = df[c].fillna(0).astype("int64")

df.head()

,scraped_at,username,video_id,caption,posted_date_time,duration_s,view_count,like_count,comment_count,repost_count,hashtags,hashtags_list
0,2026-02-02 03:12:20.410,visitdubai,7601573860256894228,our beautiful beautiful city #Dubai #VisitDuba...,2026-01-31 17:09:00,21.0,1276,98,6,3,"dubai,visitdubai,fyp","[dubai, visitdubai, fyp]"
91,2026-02-02 03:12:20.410,visitbritain,7590782903961799958,"Avengers, assemble in Norfolk! 🦸‍♀️ Join Meli...",2026-01-08 18:00:00,18.0,944,96,1,5,"starringgreatbritain,avengers,marvel,avengersf...","[starringgreatbritain, avengers, marvel, aveng..."
90,2026-02-02 03:12:20.410,visitbritain,7590783895868673302,"Join Charlie as he dives into the drama, the h...",2026-01-10 15:00:00,50.0,599,53,3,0,"rabycastle,castlebritain,durham,britainhistory","[rabycastle, castlebritain, durham, britainhis..."
89,2026-02-02 03:12:20.410,visitbritain,7590777080816700694,"Off-grid, with limited electricity and no Wi-F...",2026-01-11 15:00:00,10.0,428,29,0,2,NaN,[]
88,2026-02-02 03:12:20.410,visitbritain,7592675902580854038,"A real-life location from a legendary TV show,...",2026-01-13 18:00:00,9.0,603,23,6,1,"starringgreatbritain,filmtourism,birmingham,bl...","[starringgreatbritain, filmtourism, birmingham..."


In [101]:
### GEO NAMES ###

import re
from unidecode import unidecode
from flashtext import KeywordProcessor

CITIES_PATH = "geonames/cities15000.txt"
COUNTRIES_PATH = "geonames/countryInfo.txt"

def norm(s: str) -> str:
    """Normalize text: lowercase, remove accents, keep alnum/spaces/#/_."""
    s = unidecode(str(s).lower())
    s = re.sub(r"[^a-z0-9\s#_]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def load_cities(cities_path: str) -> pd.DataFrame:
    # GeoNames cities15000 format (tab-delimited, no header)
    cols = [
        "geonameid","name","asciiname","alternatenames","latitude","longitude",
        "feature_class","feature_code","country_code","cc2","admin1_code","admin2_code",
        "admin3_code","admin4_code","population","elevation","dem","timezone","modification_date"
    ]
    dfc = pd.read_csv(cities_path, sep="\t", names=cols, dtype=str, header=None)
    dfc["population"] = pd.to_numeric(dfc["population"], errors="coerce").fillna(0).astype(int)
    return dfc

def load_countries(country_info_path: str) -> pd.DataFrame:
    # countryInfo.txt is tab-delimited with comment lines starting with #
    rows = []
    with open(country_info_path, "r", encoding="utf-8") as f:
        for line in f:
            if not line.strip() or line.startswith("#"):
                continue
            parts = line.rstrip("\n").split("\t")
            rows.append(parts)
    # columns per GeoNames docs
    cols = [
        "ISO","ISO3","ISO_Numeric","fips","Country","Capital","Area","Population","Continent",
        "tld","CurrencyCode","CurrencyName","Phone","PostalCodeFormat","PostalCodeRegex",
        "Languages","geonameid","neighbours","EquivalentFipsCode"
    ]
    dfcty = pd.DataFrame(rows, columns=cols)
    return dfcty

def build_city_matcher(dfc: pd.DataFrame, min_pop: int = 50000) -> tuple[KeywordProcessor, dict]:
    """
    Build FlashText matcher mapping MANY names -> canonical city label.
    Canonical label: "City, CC" (e.g. "Dubai, AE") to reduce ambiguity.
    Returns matcher and canonical->metadata map.
    """
    kp = KeywordProcessor(case_sensitive=False)
    canonical_meta = {}

    df_use = dfc[dfc["population"] >= min_pop].copy()

    for _, r in df_use.iterrows():
        name = "" if pd.isna(r["name"]) else str(r["name"])
        asciiname = "" if pd.isna(r["asciiname"]) else str(r["asciiname"])
        alt = "" if pd.isna(r["alternatenames"]) else str(r["alternatenames"])
        cc = "" if pd.isna(r["country_code"]) else str(r["country_code"]).strip()

        canonical = f"{name}, {cc}" if cc else name
        canonical_meta[canonical] = {
            "geonameid": r["geonameid"],
            "name": name,
            "country_code": cc,
            "population": int(r["population"]),
        }

        # Add normalized variants
        variants = set()

        for v in [name, asciiname]:
            v = norm(v)
            if v:
                variants.add(v)

        # alternatenames is comma-separated
        for v in alt.split(","):
            v = norm(v)
            if v:
                variants.add(v)

        # Add into FlashText
        for v in variants:
            # Avoid very short tokens (noise)
            if len(v) < 3:
                continue
            kp.add_keyword(v, canonical)

    return kp, canonical_meta

def build_country_matcher(dfcty: pd.DataFrame) -> KeywordProcessor:
    """
    Build FlashText matcher for countries.
    Canonical: Country name (GeoNames 'Country' column).
    """
    kp = KeywordProcessor(case_sensitive=False)

    for _, r in dfcty.iterrows():
        country = r["Country"]
        iso2 = r["ISO"]
        iso3 = r["ISO3"]

        canonical = country

        variants = set()
        for v in [country, iso2, iso3]:
            v = norm(v)
            if v:
                variants.add(v)

        # Add "united states"/"u s a" style spacing variants lightly
        # (We do NOT hardcode huge alias lists; this is tiny formatting normalization)
        for v in list(variants):
            variants.add(v.replace(" ", ""))

        for v in variants:
            if len(v) < 2:
                continue
            kp.add_keyword(v, canonical)

    # Optional tiny overrides for common non-country region labels in social text
    # (Minimal + targeted; delete if you want strictly GeoNames-only)
    overrides = {
        "great britain": "United Kingdom",
        "britain": "United Kingdom",
        "uk": "United Kingdom",
        "u k": "United Kingdom",
        "united kingdom": "United Kingdom",
    }
    for k, v in overrides.items():
        kp.add_keyword(norm(k), v)
        kp.add_keyword(norm(k).replace(" ", ""), v)

    return kp

# Load + build matchers
df_cities = load_cities(CITIES_PATH)
df_countries = load_countries(COUNTRIES_PATH)

city_kp, city_meta = build_city_matcher(df_cities, min_pop=50000)   # tune threshold
country_kp = build_country_matcher(df_countries)

len(city_meta), df_countries.shape

(11788, (252, 19))

In [103]:
# Build full text from available cols
text_cols = [c for c in ["caption", "description", "title"] if c in df.columns]
df["full_text"] = (
    df[text_cols].fillna("").astype(str).agg(" ".join, axis=1)
    if text_cols else ""
)

if "hashtags_list" not in df.columns and "hashtags" in df.columns:
    df["hashtags_list"] = (
        df["hashtags"].fillna("").astype(str).str.lower()
        .str.replace(r"\s+", "", regex=True)
        .str.split(",")
        .apply(lambda lst: [t for t in lst if t])
    )


In [105]:
# create a new columns for locations

import re

def extract_hashtag_tokens_from_text(text: str) -> list[str]:
    return re.findall(r"#([a-z0-9_]+)", norm(text))

def combine_tokens(row):
    toks = []
    if "hashtags_list" in row and isinstance(row["hashtags_list"], list):
        toks.extend(row["hashtags_list"])
    toks.extend(extract_hashtag_tokens_from_text(row.get("full_text", "")))
    return toks

def location_extract_row(row):
    full_text = norm(row.get("full_text", ""))

    tokens = combine_tokens(row)

    # Build hashtag phrase candidates (handles visitdubai, starringgreatbritain)
    phrases = set()
    for tok in tokens:
        split_words = wordninja.split(norm(tok).replace("#","").replace("_",""))
        # add n-grams up to length 4
        for i in range(len(split_words)):
            for n in range(1, 5):
                j = i + n
                if j <= len(split_words):
                    phrase = " ".join(split_words[i:j])
                    phrases.add(phrase)
                    phrases.add(phrase.replace(" ", ""))

        # raw token variants too
        nt = norm(tok)
        phrases.add(nt)
        phrases.add(nt.replace(" ", ""))

    # Match
    countries = set(country_kp.extract_keywords(full_text))
    cities = set(city_kp.extract_keywords(full_text))

    for p in phrases:
        countries.update(country_kp.extract_keywords(p))
        cities.update(city_kp.extract_keywords(p))

    # light noise control: if too many cities, keep top 10 by pop
    if len(cities) > 10:
        cities = set(sorted(cities, key=lambda c: city_meta.get(c, {}).get("population", 0), reverse=True)[:10])

    cities = sorted(cities)
    countries = sorted(countries)
    locations = sorted(set(cities) | set(countries))

    return cities, countries, locations

df["cities_mentioned"], df["countries_mentioned"], df["locations_mentioned"] = zip(
    *df.apply(location_extract_row, axis=1)
)


In [106]:
# validate

df.explode("locations_mentioned")["locations_mentioned"].value_counts().head(25)


locations_mentioned
Teresina, BR            3120
India                   2884
Tonga                   2805
Andorra                 2281
Fortaleza, BR           1887
Italy                   1629
Iceland                 1574
Malaysia                 989
Austria                  936
Dominican Republic       801
Thika, KE                746
Somalia                  718
United Arab Emirates     714
Canada                   697
Arecibo, PR              668
Caen, FR                 610
Belgium                  598
Dayton, US               556
Montenegro               545
Buta, CD                 469
Washington, US           467
Reunion                  463
American Samoa           462
Netherlands Antilles     456
London, GB               452
Name: count, dtype: int64

In [ ]:
# 1. Does the column exist?
df.columns

